In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import math
import random

import torch
import torch.nn.functional as F
import torchvision.models as models
from torch import nn, optim
from torch.autograd import Variable
from torch.utils import model_zoo
from torch.utils.data import ConcatDataset, DataLoader, Subset
from torch.utils.tensorboard import SummaryWriter
from torchvision import datasets, transforms
from torchvision.models.resnet import BasicBlock


In [3]:

class AlexNet(nn.Module):
    def __init__(self, classes=100, dropout: float = 0.5) -> None:
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.avgpool = nn.AdaptiveAvgPool2d((6, 6))
        self.classifier = nn.Sequential(
            nn.Dropout(p=dropout),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(p=dropout),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, classes),
        )

    def forward(self, x, gt=None, flag=None, epoch=None) -> torch.Tensor:
        x = self.features(x)

        if flag:
            interval = 10
            if epoch % interval == 0:
                self.pecent = 3.0 / 10 + (epoch / interval) * 2.0 / 10

            self.eval()
            x_new = x.clone().detach()
            x_new = Variable(x_new.data, requires_grad=True)
            x_new_view = self.avgpool(x_new)
            x_new_view = x_new_view.view(x_new_view.size(0), -1)
            output = self.classifier(x_new_view)
            class_num = output.shape[1]
            index = gt
            num_rois = x_new.shape[0]
            num_channel = x_new.shape[1]
            H = x_new.shape[2]
            HW = x_new.shape[2] * x_new.shape[3]
            one_hot = torch.zeros((1), dtype=torch.float32).cuda()
            one_hot = Variable(one_hot, requires_grad=False)
            sp_i = torch.ones([2, num_rois]).long()
            sp_i[0, :] = torch.arange(num_rois)
            sp_i[1, :] = index
            sp_v = torch.ones([num_rois])
            one_hot_sparse = torch.sparse.FloatTensor(sp_i, sp_v, torch.Size([num_rois, class_num])).to_dense().cuda()
            one_hot_sparse = Variable(one_hot_sparse, requires_grad=False)
            one_hot = torch.sum(output * one_hot_sparse)
            self.zero_grad()
            one_hot.backward()
            grads_val = x_new.grad.clone().detach()
            grad_channel_mean = torch.mean(grads_val.view(num_rois, num_channel, -1), dim=2)
            channel_mean = grad_channel_mean
            grad_channel_mean = grad_channel_mean.view(num_rois, num_channel, 1, 1)
            spatial_mean = torch.sum(x_new * grad_channel_mean, 1)
            spatial_mean = spatial_mean.view(num_rois, HW)
            self.zero_grad()

            choose_one = random.randint(0, 9)
            if choose_one <= 4:
                # ---------------------------- spatial -----------------------
                mask_all = self.spatial_RCS(num_rois, H, HW, spatial_mean)
            else:
                # -------------------------- channel ----------------------------
                mask_all = self.channel_RCS(num_rois, num_channel, channel_mean)

            # ----------------------------------- batch ----------------------------------------
            cls_prob_before = F.softmax(output, dim=1)
            x_new_view_after = x_new * mask_all
            x_new_view_after = self.avgpool(x_new_view_after)
            x_new_view_after = x_new_view_after.view(x_new_view_after.size(0), -1)
            x_new_view_after = self.classifier(x_new_view_after)
            cls_prob_after = F.softmax(x_new_view_after, dim=1)

            sp_i = torch.ones([2, num_rois]).long()
            sp_i[0, :] = torch.arange(num_rois)
            sp_i[1, :] = index
            sp_v = torch.ones([num_rois])
            one_hot_sparse = torch.sparse.FloatTensor(sp_i, sp_v, torch.Size([num_rois, class_num])).to_dense().cuda()
            before_vector = torch.sum(one_hot_sparse * cls_prob_before, dim=1)
            after_vector = torch.sum(one_hot_sparse * cls_prob_after, dim=1)
            change_vector = before_vector - after_vector - 0.0001
            change_vector = torch.where(change_vector > 0, change_vector, torch.zeros(change_vector.shape).cuda())
            th_fg_value = torch.sort(change_vector, dim=0, descending=True)[0][int(round(float(num_rois) * self.pecent))]
            drop_index_fg = change_vector.gt(th_fg_value).long()
            ignore_index_fg = 1 - drop_index_fg
            not_01_ignore_index_fg = ignore_index_fg.nonzero()[:, 0]
            mask_all[not_01_ignore_index_fg.long(), :] = 1

            self.train()
            mask_all = Variable(mask_all, requires_grad=True)
            x = x * mask_all

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

    def channel_RCS(self, num_rois, num_channel, channel_mean):
        vector_thresh_percent = math.ceil(num_channel * 1 / 3.2)
        vector_thresh_value = torch.sort(channel_mean, dim=1, descending=True)[0][:, vector_thresh_percent]
        vector_thresh_value = vector_thresh_value.view(num_rois, 1).expand(num_rois, num_channel)
        vector = torch.where(channel_mean > vector_thresh_value,
                                     torch.zeros(channel_mean.shape).cuda(),
                                     torch.ones(channel_mean.shape).cuda())
        mask_all = vector.view(num_rois, num_channel, 1, 1)
        return mask_all

    def spatial_RCS(self, num_rois, H, HW, spatial_mean):
        spatial_drop_num = math.ceil(HW * 1 / 3.0)
        th18_mask_value = torch.sort(spatial_mean, dim=1, descending=True)[0][:, spatial_drop_num]
        th18_mask_value = th18_mask_value.view(num_rois, 1).expand(num_rois, 36)
        mask_all_cuda = torch.where(spatial_mean > th18_mask_value, torch.zeros(spatial_mean.shape).cuda(),
                                            torch.ones(spatial_mean.shape).cuda())
        mask_all = mask_all_cuda.reshape(num_rois, H, H).view(num_rois, 1, H, H)
        return mask_all

def alex(pretrained=True, **kwargs):
    """
    Constructs a custom alexnet model.
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
    """
    model = AlexNet(**kwargs)
    if pretrained:
        # Load the original AlexNet
        alexnet_original = models.alexnet(pretrained=True)

        # Copy weights for the common layers
        # Features
        for layer, param in model.features.named_parameters():
            param.data = alexnet_original.features.state_dict()[layer].data

        # Classifier (except for the final layer)
        for layer, param in model.classifier.named_parameters():
            if '6' not in layer:  # Skip the final layer
                param.data = alexnet_original.classifier.state_dict()[layer].data

    return model

In [4]:
class ResNet(nn.Module):
    def __init__(self, block, layers, jigsaw_classes=1000, classes=100):
        self.inplanes = 64
        super(ResNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3,
                               bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2)
        self.avgpool = nn.AvgPool2d(7, stride=1)
        # self.jigsaw_classifier = nn.Linear(512 * block.expansion, jigsaw_classes)
        self.class_classifier = nn.Linear(512 * block.expansion, classes)
        #self.domain_classifier = nn.Linear(512 * block.expansion, domains)
        self.pecent = 1/3

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

    def _make_layer(self, block, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes * block.expansion,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes * block.expansion
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes))

        return nn.Sequential(*layers)

    def is_patch_based(self):
        return False

    def forward(self, x, gt=None, flag=None, epoch=None):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        if flag:
            interval = 10
            if epoch % interval == 0:
                self.pecent = 3.0 / 10 + (epoch / interval) * 2.0 / 10

            self.eval()
            x_new = x.clone().detach() # 128，512，7，7
            x_new = Variable(x_new.data, requires_grad=True)
            x_new_view = self.avgpool(x_new) # 128 512 1 1
            x_new_view = x_new_view.view(x_new_view.size(0), -1)
            output = self.class_classifier(x_new_view)
            class_num = output.shape[1]
            index = gt
            num_rois = x_new.shape[0]
            num_channel = x_new.shape[1]
            H = x_new.shape[2]
            HW = x_new.shape[2] * x_new.shape[3]
            one_hot = torch.zeros((1), dtype=torch.float32).cuda() #
            one_hot = Variable(one_hot, requires_grad=False)
            sp_i = torch.ones([2, num_rois]).long()
            sp_i[0, :] = torch.arange(num_rois)
            sp_i[1, :] = index # location
            sp_v = torch.ones([num_rois]) # value
            one_hot_sparse = torch.sparse.FloatTensor(sp_i, sp_v, torch.Size([num_rois, class_num])).to_dense().cuda()
            one_hot_sparse = Variable(one_hot_sparse, requires_grad=False)
            one_hot = torch.sum(output * one_hot_sparse) # output-logits; class score: good bad
            self.zero_grad()
            one_hot.backward()
            grads_val = x_new.grad.clone().detach() # d(class score)/dx_new [128,512,7,7]
            grad_channel_mean = torch.mean(grads_val.view(num_rois, num_channel, -1), dim=2) # 128,512,49
            channel_mean = grad_channel_mean
            grad_channel_mean = grad_channel_mean.view(num_rois, num_channel, 1, 1)# grad_channel_mean [128 512 1 1] -->copy49 [128 512 7 7]
            spatial_mean = torch.sum(x_new * grad_channel_mean, 1) # x_new [128,512,7,7] # grad_channel_mean [128 512 1 1]
            spatial_mean = spatial_mean.view(num_rois, HW)
            self.zero_grad()

            choose_one = random.randint(0, 9)
            if choose_one <= 4:
                # ---------------------------- spatial -----------------------
                mask_all = self.spatial_RCS(num_rois, H, HW, spatial_mean)
            else:
                # -------------------------- channel ----------------------------
                mask_all = self.channel_RCS(num_rois, num_channel, channel_mean)

            # ----------------------------------- batch ----------------------------------------
            cls_prob_before = F.softmax(output, dim=1)
            x_new_view_after = x_new * mask_all
            x_new_view_after = self.avgpool(x_new_view_after)
            x_new_view_after = x_new_view_after.view(x_new_view_after.size(0), -1)
            x_new_view_after = self.class_classifier(x_new_view_after)
            cls_prob_after = F.softmax(x_new_view_after, dim=1)

            sp_i = torch.ones([2, num_rois]).long()
            sp_i[0, :] = torch.arange(num_rois)
            sp_i[1, :] = index
            sp_v = torch.ones([num_rois])
            one_hot_sparse = torch.sparse.FloatTensor(sp_i, sp_v, torch.Size([num_rois, class_num])).to_dense().cuda()
            before_vector = torch.sum(one_hot_sparse * cls_prob_before, dim=1)
            after_vector = torch.sum(one_hot_sparse * cls_prob_after, dim=1)
            change_vector = before_vector - after_vector - 0.0001
            change_vector = torch.where(change_vector > 0, change_vector, torch.zeros(change_vector.shape).cuda())
            th_fg_value = torch.sort(change_vector, dim=0, descending=True)[0][int(round(float(num_rois) * self.pecent))]
            drop_index_fg = change_vector.gt(th_fg_value).long()
            ignore_index_fg = 1 - drop_index_fg
            not_01_ignore_index_fg = ignore_index_fg.nonzero()[:, 0]
            mask_all[not_01_ignore_index_fg.long(), :] = 1

            self.train()
            mask_all = Variable(mask_all, requires_grad=True)
            x = x * mask_all

        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        return self.class_classifier(x)

    def channel_RCS(self, num_rois, num_channel, channel_mean):
        vector_thresh_percent = math.ceil(num_channel * 1 / 3.2)
        vector_thresh_value = torch.sort(channel_mean, dim=1, descending=True)[0][:, vector_thresh_percent]
        vector_thresh_value = vector_thresh_value.view(num_rois, 1).expand(num_rois, num_channel)
        vector = torch.where(channel_mean > vector_thresh_value,
                                     torch.zeros(channel_mean.shape).cuda(),
                                     torch.ones(channel_mean.shape).cuda())
        mask_all = vector.view(num_rois, num_channel, 1, 1)
        return mask_all

    def spatial_RCS(self, num_rois, H, HW, spatial_mean):
        spatial_drop_num = math.ceil(HW * 1 / 3.0)
        th18_mask_value = torch.sort(spatial_mean, dim=1, descending=True)[0][:, spatial_drop_num]
        th18_mask_value = th18_mask_value.view(num_rois, 1).expand(num_rois, 49)
        mask_all_cuda = torch.where(spatial_mean > th18_mask_value, torch.zeros(spatial_mean.shape).cuda(),
                                            torch.ones(spatial_mean.shape).cuda())
        mask_all = mask_all_cuda.reshape(num_rois, H, H).view(num_rois, 1, H, H)
        return mask_all

def resnet18(pretrained=True, **kwargs):
    """Constructs a ResNet-18 model.
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
    """
    model = ResNet(BasicBlock, [2, 2, 2, 2], **kwargs)
    if pretrained:
        model.load_state_dict(model_zoo.load_url('https://download.pytorch.org/models/resnet18-5c106cde.pth'), strict=False)
    return model


# With domain adaption Office home dataset

In [6]:


def get_model(model_name, pretrained=True, **kwargs):
    if model_name == 'resnet':
        model_class = resnet18
    elif model_name == 'alexnet':
        model_class = alex
    else:
        raise ValueError(f"Unsupported model name: {model_name}")

    return model_class(pretrained=pretrained, **kwargs)

#Accelerate the training
torch.backends.cudnn.benchmark = True
torch.manual_seed(0)
torch.cuda.manual_seed(0)


print("Target domain: {}".format('Real world'))

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Initialize the model
n_classes = 65
model = get_model('resnet', pretrained=True, classes=n_classes)
model = model.to(device)

# Data loaders
train_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.RandomResizedCrop(size=(224, 224), scale=(0.8, 1.0)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.4),
    transforms.RandomGrayscale(0.1),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])
val_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize(size=(224,224)),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])
target_dataset = datasets.ImageFolder(
    f'/content/drive/MyDrive/MLDL/Final Project/OfficeHomeDataset_10072016/Real World',
    transform=val_transform)
target_loader = torch.utils.data.DataLoader(target_dataset , batch_size=64, shuffle=True)

concatenated_datasets = []
total_size = 0

for name in ["Art","Clipart","Product"]:
    dataset = datasets.ImageFolder(
        f'/content/drive/MyDrive/MLDL/Final Project/OfficeHomeDataset_10072016/{name}',
        transform=train_transform
    )
    total_size += len(dataset)
    concatenated_datasets.append(dataset)

combined_dataset = ConcatDataset(concatenated_datasets)
indices = list(range(total_size))
random.shuffle(indices)
random_subset = Subset(combined_dataset, indices)
source_loader = DataLoader(random_subset, batch_size=64, shuffle=True)



# Optimizer and scheduler

params = model.parameters()

optimizer = optim.SGD(params, weight_decay=.0005, momentum=.9, nesterov=False, lr=0.01)
#optimizer = optim.Adam(params, lr=lr)
step_size = int(20 * .8)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=step_size)

print("Step size: %d" % step_size)

# Training and testing loop
results = {"test": torch.zeros(20)}
criterion = nn.CrossEntropyLoss()

writer = SummaryWriter('test')
global_step = 0

for now_epoch in range(20):
    scheduler.step()
    lr = scheduler.get_lr()[0]
    writer.add_scalar('Learning Rate', lr, now_epoch)

    # Training
    model.train()


    for it, batch in enumerate(source_loader):
        data, labels = batch

        data, labels = data.to(device), labels.to(device)
        optimizer.zero_grad()

        data_fliped = torch.flip(data, (3,)).detach().clone()
        data = torch.cat((data, data_fliped))
        labels_labels_flip = torch.cat((labels, labels))

        class_logit = model(data, labels_labels_flip, True, now_epoch)
        class_loss = criterion(class_logit, labels_labels_flip)
        _, cls_pred = class_logit.max(dim=1)
        loss = class_loss

        loss.backward()
        optimizer.step()

        class_loss_train = class_loss.item()
        accuracy = torch.sum(cls_pred == labels_labels_flip.data).item() / data.shape[0]
        writer.add_scalar('Loss/Train', class_loss_train, global_step)
        writer.add_scalar('Accuracy/Train', accuracy, global_step)
        global_step += 1

        print(f"Training - running batch {it}/{len(source_loader)} of epoch {now_epoch}/{20} - "
                f"loss: {class_loss_train} - acc: {accuracy * 100 : 2f}%")
        del loss, class_loss, class_logit

    # Testing
    model.eval()
    with torch.no_grad():
        class_correct = 0
        total = len(target_loader.dataset)
        for it, batch in enumerate(target_loader):
            data, labels = batch

            data, labels = data.to(device), labels.to(device)

            class_logit = model(data, labels, False)
            _, cls_pred = class_logit.max(dim=1)

            class_correct += torch.sum(cls_pred == labels.data)

        class_acc = float(class_correct) / total
        writer.add_scalar('Test/Accuracy', class_acc, now_epoch)
        print(f"Testing - acc: {class_acc : 2f}%")
        results['test'][now_epoch] = class_acc

# Output results
test_res = results["test"]
idx_best = test_res.argmax()
print(f"Best test {test_res.max()}, corresponding test {test_res[idx_best]} - best test: {test_res.max()}, best epoch: {idx_best}")

# Save the model
model_path = './model_domain_adaption_last.pth'
torch.save(model.state_dict(), model_path)
print(f"Model saved at {model_path}")

Target domain: Real world
Step size: 16


/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:136: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "
/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:384: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "
/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), e

Training - running batch 0/176 of epoch 0/20 - loss: 5.453074932098389 - acc:  0.000000%
Training - running batch 1/176 of epoch 0/20 - loss: 5.686984539031982 - acc:  0.000000%
Training - running batch 2/176 of epoch 0/20 - loss: 5.615288734436035 - acc:  0.000000%
Training - running batch 3/176 of epoch 0/20 - loss: 5.542474269866943 - acc:  0.000000%
Training - running batch 4/176 of epoch 0/20 - loss: 5.4631876945495605 - acc:  0.000000%


KeyboardInterrupt: ignored

# With domain adaption PACS

In [ ]:


def get_model(model_name, pretrained=True, **kwargs):
    if model_name == 'resnet':
        model_class = resnet18
    elif model_name == 'alexnet':
        model_class = alex
    else:
        raise ValueError(f"Unsupported model name: {model_name}")

    return model_class(pretrained=pretrained, **kwargs)

#Accelerate the training
torch.backends.cudnn.benchmark = True
torch.manual_seed(0)
torch.cuda.manual_seed(0)


print("Target domain: {}".format('sketch'))

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Initialize the model
n_classes = 7
model = get_model('resnet', pretrained=True, classes=n_classes)
model = model.to(device)

# Data loaders
train_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.RandomResizedCrop(size=(224, 224), scale=(0.8, 1.0)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.4),
    transforms.RandomGrayscale(0.1),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])
val_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize(size=(224,224)),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])
target_dataset = datasets.ImageFolder(
    f'./Domain_Generalization/PACS/kfold/sketch',
    transform=val_transform)
target_loader = torch.utils.data.DataLoader(target_dataset , batch_size=64, shuffle=True)

concatenated_datasets = []
total_size = 0

for name in ['art_painting', 'cartoon', 'photo']:
    dataset = datasets.ImageFolder(
        f'./Domain_Generalization/PACS/kfold/{name}',
        transform=train_transform
    )
    total_size += len(dataset)
    concatenated_datasets.append(dataset)

combined_dataset = ConcatDataset(concatenated_datasets)
indices = list(range(total_size))
random.shuffle(indices)
random_subset = Subset(combined_dataset, indices)
source_loader = DataLoader(random_subset, batch_size=64, shuffle=True)



# Optimizer and scheduler

params = model.parameters()

optimizer = optim.SGD(params, weight_decay=.0005, momentum=.9, nesterov=False, lr=0.01)
#optimizer = optim.Adam(params, lr=lr)
step_size = int(20 * .8)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=step_size)

print("Step size: %d" % step_size)

# Training and testing loop
results = {"test": torch.zeros(20)}
criterion = nn.CrossEntropyLoss()

writer = SummaryWriter('test_PACS')
global_step = 0

for now_epoch in range(20):
    scheduler.step()
    lr = scheduler.get_lr()[0]
    writer.add_scalar('Learning Rate', lr, now_epoch)

    # Training
    model.train()


    for it, batch in enumerate(source_loader):
        data, labels = batch

        data, labels = data.to(device), labels.to(device)
        optimizer.zero_grad()

        data_fliped = torch.flip(data, (3,)).detach().clone()
        data = torch.cat((data, data_fliped))
        labels_labels_flip = torch.cat((labels, labels))

        class_logit = model(data, labels_labels_flip, True, now_epoch)
        class_loss = criterion(class_logit, labels_labels_flip)
        _, cls_pred = class_logit.max(dim=1)
        loss = class_loss

        loss.backward()
        optimizer.step()

        class_loss_train = class_loss.item()
        accuracy = torch.sum(cls_pred == labels_labels_flip.data).item() / data.shape[0]
        writer.add_scalar('Loss/Train', class_loss_train, global_step)
        writer.add_scalar('Accuracy/Train', accuracy, global_step)
        global_step += 1

        print(f"Training - running batch {it}/{len(source_loader)} of epoch {now_epoch}/{20} - "
                f"loss: {class_loss_train} - acc: {accuracy * 100 : 2f}%")
        del loss, class_loss, class_logit

    # Testing
    model.eval()
    with torch.no_grad():
        class_correct = 0
        total = len(target_loader.dataset)
        for it, batch in enumerate(target_loader):
            data, labels = batch

            data, labels = data.to(device), labels.to(device)

            class_logit = model(data, labels, False)
            _, cls_pred = class_logit.max(dim=1)

            class_correct += torch.sum(cls_pred == labels.data)

        class_acc = float(class_correct) / total
        writer.add_scalar('Test/Accuracy', class_acc, now_epoch)
        print(f"Testing - acc: {class_acc : 2f}%")
        results['test'][now_epoch] = class_acc

# Output results
test_res = results["test"]
idx_best = test_res.argmax()
print(f"Best test {test_res.max()}, corresponding test {test_res[idx_best]} - best test: {test_res.max()}, best epoch: {idx_best}")

# Save the model
model_path = './model_PACS_domain_adaption_last.pth'
torch.save(model.state_dict(), model_path)
print(f"Model saved at {model_path}")

# With domain adaption 16 class imagenet dataset

In [ ]:


def get_model(model_name, pretrained=True, **kwargs):
    if model_name == 'resnet':
        model_class = resnet18
    elif model_name == 'alexnet':
        model_class = alex
    else:
        raise ValueError(f"Unsupported model name: {model_name}")

    return model_class(pretrained=pretrained, **kwargs)

#Accelerate the training
torch.backends.cudnn.benchmark = True
torch.manual_seed(0)
torch.cuda.manual_seed(0)

print("Target domain: {}".format('edge'))

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Initialize the model
n_classes = 7
model = get_model('resnet', pretrained=True, classes=n_classes)
model = model.to(device)

# Data loaders
train_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.RandomResizedCrop(size=(224, 224), scale=(0.8, 1.0)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.4),
    transforms.RandomGrayscale(0.1),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])
val_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize(size=(224,224)),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])
target_dataset = datasets.ImageFolder(
    f'./Domain_Generalization16_class_imagenet_datasets/edge',
    transform=val_transform)
target_loader = torch.utils.data.DataLoader(target_dataset , batch_size=64, shuffle=True)

concatenated_datasets = []
total_size = 0

for name in ['silhouette', 'cue-conflict']:
    dataset = datasets.ImageFolder(
        f'./Domain_Generalization/16_class_imagenet_datasets/{name}',
        transform=train_transform
    )
    total_size += len(dataset)
    concatenated_datasets.append(dataset)

combined_dataset = ConcatDataset(concatenated_datasets)
indices = list(range(total_size))
random.shuffle(indices)
random_subset = Subset(combined_dataset, indices)
source_loader = DataLoader(random_subset, batch_size=64, shuffle=True)



# Optimizer and scheduler

params = model.parameters()

optimizer = optim.SGD(params, weight_decay=.0005, momentum=.9, nesterov=False, lr=0.01)
#optimizer = optim.Adam(params, lr=lr)
step_size = int(20 * .8)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=step_size)

print("Step size: %d" % step_size)

# Training and testing loop
results = {"test": torch.zeros(20)}
criterion = nn.CrossEntropyLoss()

writer = SummaryWriter('test_PACS')
global_step = 0

for now_epoch in range(20):
    scheduler.step()
    lr = scheduler.get_lr()[0]
    writer.add_scalar('Learning Rate', lr, now_epoch)

    # Training
    model.train()


    for it, batch in enumerate(source_loader):
        data, labels = batch

        data, labels = data.to(device), labels.to(device)
        optimizer.zero_grad()

        data_fliped = torch.flip(data, (3,)).detach().clone()
        data = torch.cat((data, data_fliped))
        labels_labels_flip = torch.cat((labels, labels))

        class_logit = model(data, labels_labels_flip, True, now_epoch)
        class_loss = criterion(class_logit, labels_labels_flip)
        _, cls_pred = class_logit.max(dim=1)
        loss = class_loss

        loss.backward()
        optimizer.step()

        class_loss_train = class_loss.item()
        accuracy = torch.sum(cls_pred == labels_labels_flip.data).item() / data.shape[0]
        writer.add_scalar('Loss/Train', class_loss_train, global_step)
        writer.add_scalar('Accuracy/Train', accuracy, global_step)
        global_step += 1

        print(f"Training - running batch {it}/{len(source_loader)} of epoch {now_epoch}/{20} - "
                f"loss: {class_loss_train} - acc: {accuracy * 100 : 2f}%")
        del loss, class_loss, class_logit

    # Testing
    model.eval()
    with torch.no_grad():
        class_correct = 0
        total = len(target_loader.dataset)
        for it, batch in enumerate(target_loader):
            data, labels = batch

            data, labels = data.to(device), labels.to(device)

            class_logit = model(data, labels, False)
            _, cls_pred = class_logit.max(dim=1)

            class_correct += torch.sum(cls_pred == labels.data)

        class_acc = float(class_correct) / total
        writer.add_scalar('Test/Accuracy', class_acc, now_epoch)
        print(f"Testing - acc: {class_acc : 2f}%")
        results['test'][now_epoch] = class_acc

# Output results
test_res = results["test"]
idx_best = test_res.argmax()
print(f"Best test {test_res.max()}, corresponding test {test_res[idx_best]} - best test: {test_res.max()}, best epoch: {idx_best}")

# Save the model
model_path = './model_PACS_domain_adaption_last.pth'
torch.save(model.state_dict(), model_path)
print(f"Model saved at {model_path}")